#**Stock Market Efficiency**
Develop a pipeline where we hit NSE Stock Market API from RAPIDAPI website each minute & send data to AWS SQS. Once 10 records are accumulated in SQS, a Lambda function will be triggered to process the data & store it in Dynamo DB with current date & time.

####**Installing Boto Library**

In [ ]:
! pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 982.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.0 MB/s eta 0:00:00


#### **Importing All Neccessary Libraries**

In [ ]:
import requests
import boto3
import time
import json

####**Connect to AWS using Boto Client**

In [ ]:
access_key='YOUR_ACCESS_KEY'
secret_access_key='YOUR_SECRET_ACCESS_KEY'
region = 'us-east-1'

#Use boto3 client to connect with S3
client = boto3.client(
    'sqs',
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_access_key,
    region_name=region
)

#### **Create SQS Queue**

In [ ]:
def create_queue():
    response = client.create_queue(
        QueueName="my-sqs-queue",
        Attributes={
            "DelaySeconds": "0",
            "VisibilityTimeout": "60",
        }
    )
    print(response)

####**Getting the Name of stock**

In [ ]:
def getName():
  ip = input("Please Enter the Stock Name")
  stock = {"symbol": ip}
  return stock

#### **Fetching Stock Data from NSE Market API**

In [ ]:
def getData(stock):
  url = "https://nse-market.p.rapidapi.com/stock_metrics"
  headers = {
	"x-rapidapi-key": "84948a2b71mshcfc40c32a46b15ep119225jsn279efff5e964",
	"x-rapidapi-host": "nse-market.p.rapidapi.com"
  }
  res = requests.get(url, headers=headers, params=stock)
  return res.json()

#### **Passing Data to AWS SQS**

In [ ]:
def sendMessage(data):
  try:
    queue = client.get_queue_url(QueueName='mysqsqueue')
    response = client.send_message(
        QueueUrl=queue['QueueUrl'],
        MessageBody=json.dumps(data)
)
    print(response)
  except Exception as e:
    print("Error in sending message \n{}".format(e))
    return response

#### **Main Function**

In [ ]:
if __name__ == "__main__":
  name = getName()
  for i in range(15):
    time.sleep(30)
    data = getData(name)
    response = sendMessage(data)
    print(response)

Please Enter the Stock Namewipro
{'MD5OfMessageBody': 'f5b35085e5fa271b43c2bde6c92853dd', 'MessageId': '16362ac6-28e0-4d4a-9312-e034928c8ef3', 'ResponseMetadata': {'RequestId': '56e98667-7e44-55d1-9b7c-d8987cba540f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '56e98667-7e44-55d1-9b7c-d8987cba540f', 'date': 'Wed, 03 Jul 2024 07:41:50 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '106', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
None
{'MD5OfMessageBody': 'f5b35085e5fa271b43c2bde6c92853dd', 'MessageId': '2d7e3f81-b652-49b4-89b4-cb9384865f5c', 'ResponseMetadata': {'RequestId': 'b4066965-d06d-5dbd-ad35-379ad370bc63', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b4066965-d06d-5dbd-ad35-379ad370bc63', 'date': 'Wed, 03 Jul 2024 07:42:21 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '106', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
None
{'MD5OfMessageBody': 'f5b35085e5fa271b43c2bde6c92853dd', 'MessageId